In [1]:
# Запуск простой программы Hello World! как Flask сервера

In [1]:
!pip install flask-ngrok
!pip install flask==0.12.2

  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)
  Using cached certifi-2021.5.30-py2.py3-none-any.whl (145 kB)
  Using cached idna-3.2-py3-none-any.whl (59 kB)
     |████████████████████████████████| 138 kB 713 kB/s eta 0:00:01


# Создаем Flask Приложение с вызовом модели

In [ ]:
# %%writefile flask_app2.py
# Загружаем билиотеки
import flask
from flask import Flask, request, redirect,url_for, send_from_directory
import pandas as pd
import requests
import json
import os
import numpy as np
from flask_ngrok import run_with_ngrok # эта строчка не нужна, если запускаете на своем ПК
from werkzeug.utils import secure_filename

# Задаем имя серверу
app = Flask(__name__)
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False
app.config['UPLOAD_FOLDER'] = '/content/'
ALLOWED_EXTENSIONS = {'png','jpeg'}
run_with_ngrok(app) # эта строчка не нужна, если запускаете на своем ПК

# Загрузка модели

def predict_model(x):
  return x ** 2 + 1 / x - np.log(x) * 0.5 * x ** 3


def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


@app.route('/uploads/<name>')
def download_file(name):
  return send_from_directory(app.config['UPLOAD_FOLDER'], name)


@app.route('/web', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            print('No file part')
            return redirect(request.url)
        file = request.files['file']
        # If the user does not select a file, the browser submits an
        # empty file without a filename.
        if file.filename == '':
            print('No selected file')
            return redirect(request.url)

        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            return redirect(url_for('download_file', name=filename))
    return '''
    <!doctype html>
    <title>Upload new File</title>
    <h1>Upload new File</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file>
      <input type=submit value=Upload>
    </form>
    '''


# Задаем функцию Predict
@app.route("/predict", methods=["GET", "POST"])
def predict():
  data = {"success": False}
  params = flask.request.json
  if (params == None):
    params = flask.request.args

	# Если параметры найдены, создаем из них Pandas DataFrame
  if (params != None):
    x = pd.DataFrame.from_dict(params, orient='index').transpose().astype('int').values

  # Записываем значение prediction в data["prediction"]
  data["prediction"] = predict_model(x)
  #Записываем статус в data["success"]
  data["success"] = True
    
  #Возвращаем результат json format 
  return flask.jsonify(data) 


@app.route("/train", methods=["POST"])
def retraining():
  x = json.loads(flask.request.json)['train_x']
  y = json.loads(flask.request.json)['train_y']
  return flask.jsonify({'x':x})


# Запускаем Сервер
if __name__ == '__main__':
    app.run() 



 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [ ]:
import requests
import json
data_all = {'train_x': x[0].reshape(-1,784).tolist(), 'train_y' : x[1].tolist()}
file = {'file': open('/content/flask_test.h5', 'rb')}
r = requests.post('http://bfbd-35-238-28-159.ngrok.io/train', json=json.dumps(data_all),files= file)

In [4]:
# импортируем библиотеку requests
import requests

# Создаем словарь dict с параметрами
data = {'key1': 0,
           'key2': 1,
          'key3': 1,
          'key4': 0,
          'key5': 1,
          'key6': 1,
          'key7': 0,
          'key8': 1,
          'key9': 1,
          'key10': 1,}

# Запрашиваем Web-сервис, передаем в запрос параметры
r = requests.get('http://127.0.0.1:5000/predict', params=data)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /predict?key1=0&key2=1&key3=1&key4=0&key5=1&key6=1&key7=0&key8=1&key9=1&key10=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f04d48202b0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
# Смотрим ответ Web-сервиса
r.text

In [13]:
!python flask_app2.py


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Running on http://9782-35-238-28-159.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


# Домашнее задание

Задание **Light**:


1.   Напишите свою функцию predict_model следующего вида: y = x**2 + 1/x - log(x) * 1/2x**3
2. Измините функцию загрузки файлов таким образом чтобы можно было сохранять csv файлы. В функции upload_file после сохранения файла прочитайте его с помощью библиотеки pandas и выведите текст первых 2-х строчек файла вместо return redirect(url_for('download_file', name=filename))

Задание **PRO**:

1. Напишите метод fit, который на вход получает два массива значение x и y и обучает простую функцию np.polyfit, после чего сохраняет картинку с графиками исходной и приближенной функции и выводит ее в браузере
см. пример кода:
import numpy as np
x = np.array([0.0, 1.0, 2.0, 3.0,  4.0,  5.0])
y = np.array([0.0, 0.8, 0.9, 0.1, -0.8, -1.0])
z = np.polyfit(x, y, 4)


import matplotlib.pyplot as plt
p = np.poly1d(z)
p30 = np.poly1d(np.polyfit(x, y, 30))
xp = np.linspace(-2, 6, 100)
_ = plt.plot(x, y, '.', xp, p(xp), '-', xp, p30(xp), '--')
plt.ylim(-2,2)
(-2, 2)
plt.show()

2. Используя библиотеку Dash создайти приложение, в котором будет построен график на основе следующих данных:
https://github.com/ashishpatel26/Introduction-to-Time-Series-forecasting/blob/master/international-airline-passengers.csv
Добавьте Slider для выбора года и DropDown для выбора месяца, напишите callback функции для перерисовки графика после перемещения Slider или выбора в Dropdown


Результаты по ДЗ предоставить в виде ссылки на Google colaboratory ноутбук